# Deep Learning model for national exam analysis

This notebook analyzes student performance.

**Author:** Birhanu Moges


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load CSV file
df = pd.read_csv(r"C:/Users/DELL/project/ethiopian_students_dataset.csv")

# View first 5 rows
print(df.head())

# Access a column
print(df.columns)

In [ ]:
print("="*60)
print("📊 DATASET OVERVIEW")
print("="*60)
print("Shape:", df.shape)
print("\nData Types & Missing Values:")
df.info()

print("\n" + "="*60)
print("📈 DESCRIPTIVE STATISTICS (NUMERIC)")
print("="*60)
display(df.describe().T)

In [ ]:
# 1. Get the value counts of dtypes
dtype_counts = df.dtypes.value_counts().reset_index()
dtype_counts.columns = ['Data Type', 'Count']

# 2. Plotting with the fix
plt.figure(figsize=(10, 6))

# Fix: Assign 'Data Type' to hue and set legend=False
sns.barplot(
    data=dtype_counts,
    x='Data Type',
    y='Count',
    hue='Data Type',
    palette='viridis',
    legend=False
)

plt.title('Distribution of Data Types in Student Dataset', fontsize=14)
plt.ylabel('Number of Columns')
plt.xlabel('Data Type')

# 3. Add labels on top of bars
for i, count in enumerate(dtype_counts['Count']):
    plt.text(i, count + 5, str(count), ha='center', fontweight='bold')

plt.show()

In [ ]:
# ================================
# 1️⃣ INITIAL CLEANING & ENCODING
# ================================
# Drop Student_ID (never used in ML)
df = df.drop(columns=['Student_ID'], errors='ignore')

# Encode Field_Choice (Social=0, Natural=1)
df['Field_Choice'] = df['Field_Choice'].map({'Social': 0, 'Natural': 1})

# Fill missing Career_Interest with "Unknown"
df['Career_Interest'] = df['Career_Interest'].fillna('Unknown')


# ================================
# 2️⃣ DEFINE EDUCATION STAGES
# ================================
lower_primary = ['Grade_1', 'Grade_2', 'Grade_3', 'Grade_4']
upper_primary = ['Grade_5', 'Grade_6', 'Grade_7', 'Grade_8']
secondary     = ['Grade_9', 'Grade_10']
preparatory   = ['Grade_11', 'Grade_12']

stages = {
    'Lower_Primary': lower_primary,
    'Upper_Primary': upper_primary,
    'Secondary': secondary,
    'Preparatory': preparatory
}


# ================================
# 3️⃣ HELPER FUNCTION TO AGGREGATE GRADES
# ================================
def stage_average(df, grades, metric_keywords):
    """
    Compute average across all columns for a given stage and metric keywords.
    Returns the average series and list of original columns used.
    """
    cols = []
    for g in grades:
        for keyword in metric_keywords:
            cols += [c for c in df.columns if c.startswith(g) and keyword.lower() in c.lower()]
    cols = list(set(cols))
    return df[cols].mean(axis=1), cols


# ================================
# 4️⃣ AGGREGATE TEST SCORE, ATTENDANCE, HW, PARTICIPATION
# ================================
metrics_dict = {
    'Test_Score': ['Test_Score'],
    'Attendance': ['Attendance'],
    'HW_Completion': ['Homework_Completion'],
    'Participation': ['Participation']
}

cols_to_drop = []

for metric_name, keywords in metrics_dict.items():
    for stage_name, grades in stages.items():
        col_name = f'Avg_{metric_name}_{stage_name}'
        df[col_name], original_cols = stage_average(df, grades, keywords)
        cols_to_drop += original_cols

# Drop original grade-level columns
df.drop(columns=list(set(cols_to_drop)), inplace=True)

# Columns list for display
aggregated_cols = [f'Avg_{m}_{s}' for m in metrics_dict.keys() for s in stages.keys()]
print("Aggregated averages per Education Stage (head):")
print(df[aggregated_cols].head())


# ================================
# 5️⃣ AGGREGATE TEXTBOOK ACCESS
# ================================
# Convert Yes/No → 1/0 safely
textbook_cols = [c for c in df.columns if 'Textbook' in c]
for col in textbook_cols:
    df[col] = df[col].replace({'Yes': 1, 'No': 0}).infer_objects(copy=False)

# Helper function for textbook access per stage
def textbook_access(df, grade_prefixes):
    cols = []
    for g in grade_prefixes:
        cols.extend([c for c in df.columns if c.startswith(g) and 'Textbook' in c])
    return df[cols].mean(axis=1) if len(cols) > 0 else pd.Series(0, index=df.index)

# Create aggregated textbook access per stage
new_cols_df = pd.DataFrame({
    'Textbook_Access_1_4': textbook_access(df, lower_primary),
    'Textbook_Access_5_8': textbook_access(df, upper_primary),
    'Textbook_Access_9_10': textbook_access(df, secondary),
    'Textbook_Access_11_12': textbook_access(df, preparatory)
})

df = pd.concat([df, new_cols_df], axis=1)
df = df.loc[:, ~df.columns.duplicated()]  # remove duplicates

# Display and visualize
textbook_summary_cols = [c for c in new_cols_df.columns if c in df.columns]
print(df[textbook_summary_cols].head())

plt.figure(figsize=(10, 6))
sns.boxplot(data=df[textbook_summary_cols])
plt.title('Textbook Access Distribution by Education Level', fontsize=14, fontweight='bold')
plt.ylabel('Access Score (0 to 1)')
plt.xticks(rotation=15)
plt.grid(alpha=0.3)
plt.show()


# ================================
# 6️⃣ TRACK-BASED NATIONAL EXAMS
# ================================
# Subjects per track
social_subjects = ['National_Exam_History', 'National_Exam_Geography',
                   'National_Exam_Economics', 'National_Exam_Math_Social']

natural_subjects = ['National_Exam_Biology', 'National_Exam_Chemistry',
                    'National_Exam_Physics', 'National_Exam_Math_Natural']

# Track-specific averages
df['Social_Track_Subject_Avg']  = df[social_subjects].mean(axis=1)
df['Natural_Track_Subject_Avg'] = df[natural_subjects].mean(axis=1)

# Track-based assignment
df['Track_Subject_Average'] = np.where(
    df['Field_Choice'] == 0,
    df['Social_Track_Subject_Avg'],
    df['Natural_Track_Subject_Avg']
)

# Common subjects for all students
common_subjects = ['National_Exam_Aptitude', 'National_Exam_English',
                   'National_Exam_Civics_and_Ethical_Education']
df['Common_Exam_Average'] = df[common_subjects].mean(axis=1)

# Overall Track Exam Average
df['Track_Exam_Average'] = (df['Common_Exam_Average'] + df['Track_Subject_Average']) / 2

# Display new exam columns
exam_cols = [
    'Social_Track_Subject_Avg',
    'Natural_Track_Subject_Avg',
    'Track_Subject_Average',
    'Common_Exam_Average',
    'Track_Exam_Average'
]
print("New Aggregated National Exam Features:")
print(df[exam_cols].head())


# ================================
# 7️⃣ VISUALIZATION: Exam Scores
# ================================
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot: Common vs Track vs Overall
sns.boxplot(data=df[['Common_Exam_Average', 'Track_Subject_Average', 'Track_Exam_Average']],
            ax=axes[0], palette="Set2")
axes[0].set_title('Distribution of Aggregate Exam Scores')
axes[0].set_ylabel('Score (0-100)')

# KDE: Track Exam Average by Field Choice
for choice, label in [(0, 'Social Science'), (1, 'Natural Science')]:
    subset = df[df['Field_Choice'] == choice]
    sns.kdeplot(subset['Track_Exam_Average'], ax=axes[1], label=label, fill=True)

axes[1].set_title('Track Exam Average: Social vs. Natural')
axes[1].set_xlabel('Score')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# DROP ORIGINAL HIGH-DIMENSION COLUMNS
drop_cols = [c for c in df.columns if c.startswith('Grade_')]
drop_cols += [c for c in df.columns if c.startswith('National_Exam_')]

df = df.drop(columns=drop_cols)
# -------------------------------
# 0️⃣ Drop leaking exam average columns
# -------------------------------
leak_cols = [
    'Social_Track_Subject_Avg',
    'Natural_Track_Subject_Avg',
    'Track_Exam_Average',
    'Track_Subject_Average',
    'Common_Exam_Average',
     'School_ID','Total_Test_Score','Overall_Average']

df = df.drop(columns=[c for c in leak_cols if c in df.columns])

# fix null value
df['Health_Issue'] = df['Health_Issue'].fillna('No Issue')
df['Father_Education'] = df['Father_Education'].fillna('Unknown')
df['Mother_Education'] = df['Mother_Education'].fillna('Unknown')

# FINAL CHECK
print(df.shape)
print(df.head())
print("all columns:",df.columns)

# Find duplicates
duplicates = df[df.duplicated()]

In [ ]:
# Select categorical columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

# Prepare a dictionary to store info
cat_summary = {}

for col in cat_cols:
    unique_vals = df[col].unique()
    cat_summary[col] = {
        'Unique_Count': len(unique_vals),
        'Unique_Values': unique_vals
    }

# Display summary in a readable way
print("Unique count and value of catagorical features:")
for col, info in cat_summary.items():
    print(f"Column: {col}")
    print(f"  Unique Count : {info['Unique_Count']}")
    print(f"  Unique Values: {info['Unique_Values']}\n")

In [ ]:
df['Date_of_Birth'] = pd.to_datetime(df['Date_of_Birth'])

df['Birth_Year'] = df['Date_of_Birth'].dt.year
df['Birth_Month'] = df['Date_of_Birth'].dt.month
df['Birth_Day'] = df['Date_of_Birth'].dt.day

df.drop(columns=['Date_of_Birth'], inplace=True)

In [ ]:
binary_cols = [
    'Gender',
    'Home_Internet_Access',
    'Electricity_Access',
    'School_Location'
]

binary_map = {
    'Yes': 1, 'No': 0,
    'Male': 1, 'Female': 0,
    'Urban': 1, 'Rural': 0
}

for col in binary_cols:
    df[col] = df[col].map(binary_map)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense

embedding_cols = [
    'Region',
    'Health_Issue',
    'Father_Education',
    'Mother_Education',
    'School_Type',
    'Career_Interest'
]
label_encoders = {}

for col in embedding_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

target = 'Total_National_Exam_Score'

X = df.drop(columns=[target])
y = df[target].values

numerical_cols = X.select_dtypes(include=['float64', 'int64']) \
                  .columns.difference(embedding_cols)

scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
inputs = []
embeddings = []

for col in embedding_cols:
    n_unique = df[col].nunique()
    embed_dim = min(50, (n_unique + 1) // 2)

    inp = Input(shape=(1,), name=col)
    emb = Embedding(
        input_dim=n_unique + 1,
        output_dim=embed_dim,
        name=f"{col}_embedding"
    )(inp)
    emb = Flatten()(emb)

    inputs.append(inp)
    embeddings.append(emb)

In [ ]:
numerical_cols = (
    X.select_dtypes(include=['float64', 'int64'])
     .columns
     .difference(embedding_cols)
)
num_input = Input(shape=(len(numerical_cols),), name="numerical")
inputs.append(num_input)

print("Numerical columns:", numerical_cols)
print("Number of numerical features:", len(numerical_cols))

In [ ]:
from tensorflow.keras.layers import BatchNormalization

x = Concatenate()(embeddings + [num_input])

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(64, activation='relu')(x)

output = Dense(1, activation='linear')(x)

model = Model(inputs=inputs, outputs=output)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.Huber(),
    metrics=['mae']
)

In [ ]:
def prepare_inputs(dataframe):
    inputs = {}
    for col in embedding_cols:
        inputs[col] = dataframe[col].values
    inputs['numerical'] = dataframe[numerical_cols].values
    return inputs

In [ ]:
train_inputs = prepare_inputs(X_train)
test_inputs = prepare_inputs(X_test)